# Hackathon III - MLOps

In previous lessonss we've already preprocess the data and created a dataset for training. In this lesson we will:

- 👉 Split the data into: Train / Test / Validation (0.5 p)
- 👉 Train a Regression Model Pipeline (1 p)
- 👉 Evaluate the model (1 p)
- 👉 Log the model and the metrics using MLflow (1 p)
- 👉 Register the model in MLflow (1 p)
- 👉 Deploy the model to a REST API (1 p)
- 👉 Make predictions using the REST API (1 p)
- 👉 Upload the predictions to the database (0.5 p)

Also the following activities will be evaluated:

- 👉 Code legibility (0.5 p)
- 👉 Notebook documentation: titles, subtitles, text explanation, etc. (0.5 p)
- 👉 Creating a GIT repository with the code:
  - 👉 Adding a complete README.md file (0.5 p)
  - 👉 Adding a .gitignore file and a requirements.txt file (0.5 p)
  - 👉 Using Branches for development (1 p)

Bonus points:

- ✅ Any other activity that the student considers important and reflected in code (ie.: GridSearch)
- ✅ Trying with other articles

Following activities will be negatively evaluated:

- ❌ Pushing Data to the repository (it's a bad practice)
- ❌ Pushing passwords or sensitive information to the repository (It's a VERY bad practice)
- ❌ Pushing files generated by MLFlow like `mlruns` (it's a bad practice)

In [1]:
# Avoid unnecessary warnings

import warnings
warnings.filterwarnings('ignore')

## Load the data

First step always is data loading. CSV are provided and can be replicated from the previows lesson. CSVs contains data for the PRODUCT_ID 3960.

- 👉 Load train and test datasets.
- 👉 Set "fecha_venta" as index column.
- 👉 Sort the data by "fecha_venta" column.
- 👉 Show the first 5 rows of the train dataset.

In [2]:
import pandas as pd

TRAIN_DATA_PATH = "data/itemSalesTrain.csv"                          #This file only includes data of the article 3960
df_train = pd.read_csv(TRAIN_DATA_PATH)             
df_train = df_train.set_index("fecha_venta").sort_index()            #Set fecha_venta as index and sort by fecha_venta column
df_train


,var1(t-7 day),var1(t-6 day),var1(t-5 day),var1(t-4 day),var1(t-3 day),var1(t-2 day),var1(t-1 day),var1(t-5 week),var1(t-4 week),var1(t-3 week),...,var1(t),tavg_w,tmin_w,tmax_w,isfestivo,day,dayofweek,month,year,weekday
fecha_venta,,,,,,,,,,,,,,,,,,,,,
2021-06-05,309.0,357.0,240.0,207.0,171.0,153.0,228.0,402.0,276.0,216.0,...,321.0,21.2,19.0,24.3,0,5,6,6,2021,0
2021-06-06,357.0,240.0,207.0,171.0,153.0,228.0,321.0,369.0,345.0,324.0,...,318.0,20.7,16.4,25.1,0,6,7,6,2021,0
2021-06-07,240.0,207.0,171.0,153.0,228.0,321.0,318.0,228.0,174.0,177.0,...,204.0,22.0,16.9,26.9,0,7,1,6,2021,1
2021-06-08,207.0,171.0,153.0,228.0,321.0,318.0,204.0,237.0,174.0,216.0,...,231.0,22.8,17.9,28.7,0,8,2,6,2021,1
2021-06-09,171.0,153.0,228.0,321.0,318.0,204.0,231.0,288.0,180.0,162.0,...,198.0,23.3,18.2,29.1,0,9,3,6,2021,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-02,321.0,183.0,156.0,207.0,192.0,216.0,210.0,507.0,429.0,276.0,...,315.0,18.5,12.7,24.5,1,2,7,4,2023,0
2023-04-03,183.0,156.0,207.0,192.0,216.0,210.0,315.0,294.0,201.0,150.0,...,210.0,15.7,9.9,24.1,1,3,1,4,2023,1
2023-04-04,156.0,207.0,192.0,216.0,210.0,315.0,210.0,486.0,168.0,219.0,...,204.0,14.9,12.2,18.5,1,4,2,4,2023,1


In [3]:
TEST_DATA_PATH = "data/itemSalesTest.csv"
df_test = pd.read_csv(TEST_DATA_PATH)
df_test = df_test.set_index("fecha_venta").sort_index()
df_test

,var1(t-7 day),var1(t-6 day),var1(t-5 day),var1(t-4 day),var1(t-3 day),var1(t-2 day),var1(t-1 day),var1(t-5 week),var1(t-4 week),var1(t-3 week),...,var1(t),tavg_w,tmin_w,tmax_w,isfestivo,day,dayofweek,month,year,weekday
fecha_venta,,,,,,,,,,,,,,,,,,,,,
2023-04-07,216.0,210.0,315.0,210.0,204.0,231.0,327.0,273.0,255.0,177.0,...,330.0,13.9,8.5,19.8,1,7,5,4,2023,1
2023-04-08,210.0,315.0,210.0,204.0,231.0,327.0,330.0,303.0,312.0,327.0,...,267.0,14.1,6.4,21.5,1,8,6,4,2023,0
2023-04-09,315.0,210.0,204.0,231.0,327.0,330.0,267.0,429.0,276.0,369.0,...,315.0,14.9,9.3,20.6,1,9,7,4,2023,0
2023-04-10,210.0,204.0,231.0,327.0,330.0,267.0,315.0,201.0,150.0,153.0,...,213.0,22.8,12.7,33.7,0,10,1,4,2023,1
2023-04-11,204.0,231.0,327.0,330.0,267.0,315.0,213.0,168.0,219.0,216.0,...,153.0,17.8,14.6,22.6,0,11,2,4,2023,1
2023-04-12,231.0,327.0,330.0,267.0,315.0,213.0,153.0,183.0,192.0,216.0,...,183.0,22.6,14.5,29.7,0,12,3,4,2023,1
2023-04-13,327.0,330.0,267.0,315.0,213.0,153.0,183.0,177.0,255.0,270.0,...,138.0,20.2,15.4,26.5,0,13,4,4,2023,1
2023-04-14,330.0,267.0,315.0,213.0,153.0,183.0,138.0,255.0,177.0,150.0,...,204.0,21.8,16.1,29.2,0,14,5,4,2023,1
2023-04-15,267.0,315.0,213.0,153.0,183.0,138.0,204.0,312.0,327.0,309.0,...,276.0,22.0,18.1,27.8,0,15,6,4,2023,0


## Split data into train and test sets

We will split the data into train and test sets. We will use the train set to train the model and the test set to evaluate it.


- 👉 Select the feature columns and the label columns
- 👉 Split the data into train and validation sets using a 80/20 or 90/10 ratio.
- 💡 Remember how train/val split should be made in time series problems.

In [4]:
# Set the product id and family (only for logging purposes, do not use it for filtering)
PRODUCT_ID = 3960
PRODUCT_FAMILY = "BOLLERIA"

df_test.columns     # To see what dependent variable we want to use (var1(t)). Then, we set the label and the features

LABEL_COLUMN = "var1(t)"

FEATURES_COLUMN = [
    'var1(t-7 day)',
    'var1(t-6 day)', 
    'var1(t-5 day)', 
    'var1(t-4 day)',
    'var1(t-3 day)', 
    'var1(t-2 day)', 
    'var1(t-1 day)', 
    'var1(t-5 week)',
    'var1(t-4 week)', 
    'var1(t-3 week)', 
    'var1(t-2 week)', 
    'var1(t-1 week)',
    'var1(t)', 
    'tavg_w', 
    'tmin_w', 
    'tmax_w', 
    'isfestivo', 
    'day',
    'dayofweek', 
    'month', 
    'year', 
    'weekday'
]

features = df_train[FEATURES_COLUMN]
labels = df_train[LABEL_COLUMN]

VAL_SPLIT = 0.1

train_size = int(len(df_train) * (1 - VAL_SPLIT))       # 90% of the data

X_train = features[:train_size]
y_train = labels[:train_size]
X_val = features[train_size:]
y_val = labels[train_size:]

## Start MLFlow Server

- 👉 Launch a local MLFlow server
- 👉 Connect to local MLFlow server
- 👉 Set the desired experiment
- 👉 Enable MLFlow autologing for sklearn

In [ ]:
%pip install mlflow

In [5]:
import mlflow

In [6]:
# Connect to the mlflow server hosted locally and setting the url in a variable
MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)        #To launch the server, we open the terminal and write "mlflow ui"

In [ ]:
# After we create a new experiment (I did it on the webpage, it´s called "Sales Forecasting"), we have to connect to the experiment.
MLFLOW_EXPERIMENT_NAME = "Sales Forecasting"
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)       # Set what the experiment is. 

In [8]:
#Set the autlog sklearn
mlflow.sklearn.autolog()

## Train and evaluate the model

The next section is to train and evaluate the model. We will use a pipeline to preprocess the data and train the model.

- 👉 Create a Sklearn Pipeline:
  - 👉 Preprocessing: StandardScaler or MinMaxScaler
  - 👉 Model: LinearRegression, RandomForestRegressor, etc.
- 👉 Start a run in MLFlow
- 👉 Train the model using the train dataset
- 👉 Add convenient tags for PRODUCT_ID and FAMILY_ID
- 👉 Evaluate the model
- 💡 Remember this is a regression problem
- 💡 Autolog will automatically log metrics and model

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics

scaler = MinMaxScaler()
model = LinearRegression()
pipe = Pipeline([
    ("scaler", scaler),
    ("model", model)
])

with mlflow.start_run() as run:                         # Start a run with these information:

    pipe.fit(X_train, y_train)

    mlflow.set_tag("product_id", PRODUCT_ID)            # Add tags in columns
    mlflow.set_tag("product_family", PRODUCT_FAMILY)

    predictions = pipe.predict(X_val)
    mae = metrics.mean_absolute_error(y_val, predictions)   # Add mae, mse and r2 metrics
    mse = metrics.mean_squared_error(y_val, predictions)
    r2 = metrics.r2_score(y_val, predictions)


## Register the model

Promote the model to Model Registry. For this section you can choose between using the MLflow UI or using code snipets. If you choose the UI you should provide screenshots.

![Model Sales Forecasting](data/model1_mlflow_production.PNG)

## Tag the Model

We can assign a tag to the model to indicate that it is ready for production. This way all versions (v1, v2...) of the model will have the same tag. So we can deploy the model by selecting the (same) tag instead of a specific (different) version.

For this section you can choose between using the MLflow UI or using code snipets. If you choose the UI you should provide screenshots.

![Model Sales Forecasting Tag "production"](data/mlflow_2.PNG)

## Deploy the model

In a terminal run the following command to deploy the model:

```bash
export MLFLOW_TRACKING_URI=http://localhost:5000
mlflow models serve -m models:/<model_name>@production -p 5001 --env-manager local
```

You should see something like this:

```bash
[INFO] Starting gunicorn 21.2.0
[INFO] Listening at: http://127.0.0.1:5001 (236041)
[INFO] Using worker: sync
[INFO] Booting worker with pid: 236048
```

It means it's working correctly 🎉

## Make requests to the model

The model is now deployed and ready to receive requests. We will make a request to the model using the test set.

- 👉 prepare the test set to be sent as JSON
- 👉 make a POST request to the model
- 👉 get the predictions from the response and show them

In [ ]:
import requests
import json


URL = "https://127.0.0.1:5001/invocations"
headers = {"Content-Type": "application/json"}

data = {
    "dataframe_split": {
        "columns": df_test.columns.tolist(),
        "index": df_test.index.to_list(),
        "data": df_test.values.tolist()
    }
}

response = requests.post(URL, headers=headers, data=json.dumps(data))
predictions = response.json()["predictions"]

## Push Results to Database

We push the results to the database so we can visualize them using other tools like Tableau, PowerBI, etc.

In [10]:
# Helpfull class used to connect to the database and push dataframes

import sqlalchemy as sa


class DatabaseConnection:

    def __init__(
        self,
        username: str,
        password: str,
        dialect: str = "mysql",
        driver: str = "pymysql",
        host: str = "database-1.cxlpff3hacbu.eu-west-3.rds.amazonaws.com",
        port: int = 3306,
        database: str = "sandbox",
    ) -> None:
        """Creates a connection to a database

        Args:
            username (str): username
            password (str): password
            dialect (str, optional): dialect. Defaults to "mysql".
            driver (str, optional): driver. Defaults to "pymysql".
            host (str, optional): host. Defaults to "database-1.crek3tiqyj7r.eu-west-3.rds.amazonaws.com".
            port (int, optional): port. Defaults to 3306.
            database (str, optional): database. Defaults to "classicmodels".
        """
        connection_string = f"{dialect}+{driver}://{username}:{password}@{host}:{port}/{database}"
        self.engine = sa.create_engine(connection_string)

    def insert_dataframe(self, df: pd.DataFrame, table_name: str) -> None:
        """Inserts a dataframe into a table
        
        Args:
            df (pd.DataFrame): dataframe to insert
            table_name (str): table name
        """
        df.to_sql(table_name, self.engine, if_exists="replace", index=False)

    def query_to_df(self, query: str) -> pd.DataFrame:
        """Retrieves a dataframe from a query.

        Args:
            query (str): query to perform.

        Returns:
            pd.DataFrame: daframe with the results of the query.
        """
        with self.engine.connect() as conn:
            df = pd.read_sql_query(query, conn)
            return df

    def check_connection(self) -> bool:
        """Checks if the connection is working

        Returns:
            bool: True if the connection is working, False otherwise
        """
        try:
            self.engine.connect()
        except Exception as e:
            print(e)
            return False

Prepare the dataframe to upload to the database

In [ ]:
dates = df_test.index.to_list()


# Create a dataframe with the data to store
df_article_prediction = pd.DataFrame({
    "fecha": dates,
    "cantidad": predictions,
    "articulo": [PRODUCT_ID] * len(predictions),    # repeat the article for each date
    "familia": [PRODUCT_FAMILY] * len(predictions),                                  # repeat the family for each date
})

Push the dataframe to the database

In [ ]:
# Load environment variables to hide credentials
import os
from dotenv import load_dotenv
load_dotenv(verbose=True)

In [ ]:
# Database credentials
USER = os.getenv("EV_user"),
PASSWORD = os.getenv("EV_password")
NAME = "david97p"
table_name = f"Materials_Prediction_Group_{NAME}"


# Connect to the database
db = DatabaseConnection(USER, PASSWORD)
db.insert_dataframe(df_article_prediction, table_name)